In [198]:
import torch
print('Version', torch.__version__)
print('CUDA enabled:', torch.cuda.is_available())

Version 1.10.0+cu102
CUDA enabled: False


In [199]:
import torch.nn as nn
from torchvision import datasets
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import tqdm

import sys
import os
import pickle
import re
import csv
import multiprocessing

import pt_util

In [200]:
DATA_DIR = 'dakshina_dataset_v1.0'
LANG = 'ta'
LANG_DIR = 'lexicons'
DATA_PATH = '{}/{}/{}'.format(DATA_DIR, LANG, LANG_DIR)

In [201]:
def prepare_data(dirpath):
    # Read train and test data
    filenames = os.listdir(dirpath)
    train_data = []
    test_data = []
    max_len = 0 # max length of any word in the dataset
    
    def get_most_annotated_transliterations(tsv_file, data, max_len):
        curr_lang = None
        max_annotations = 0
        curr_translits = {}

        # only consider the most annotated transliteration(s) for each foreign language word
        for lang, en, annotations in tsv_file:
            if not curr_lang or lang != curr_lang:
                for translit, annots in curr_translits.items():
                    if annots == max_annotations:
                        data.append([curr_lang, translit])
                        max_len = max(max_len, len(translit))
                curr_lang = lang
                max_annotations = annotations
                curr_translits = {en: annotations}
                max_len = max(max_len, len(curr_lang))
            else:
                max_annotations = max(max_annotations, annotations)
                curr_translits[en] = annotations
        for translit, annots in curr_translits.items(): # fencepost
            if annots == max_annotations:
                data.append([curr_lang, translit])
                max_len = max(max_len, len(translit))
        return max_len
    
    for filename in filenames:
        with open(os.path.join(dirpath, filename), 'r') as file:
            tsv = csv.reader(file, delimiter='\t')
            if filename.endswith('train.tsv'):
                max_len = get_most_annotated_transliterations(tsv, train_data, max_len)
            else:
                max_len = get_most_annotated_transliterations(tsv, test_data, max_len)
                
    return train_data, test_data, max_len

In [209]:
PAD = '<PAD>'
def create_dictionaries(data):
    #en_voc2ind = {PAD: 0}
    #lang_voc2ind = {PAD: 0}
    #en_ind2voc = {0: PAD}
    #lang_ind2voc = {0: PAD}
    
    en_voc2ind = {}
    lang_voc2ind = {}
    en_ind2voc = {}
    lang_ind2voc = {}
    
    en_ind = 0 #1
    lang_ind = 0 #1
    for lang, en in data:
        for lang_char in lang:
            if lang_char not in lang_voc2ind:
                lang_voc2ind[lang_char] = lang_ind
                lang_ind2voc[lang_ind] = lang_char
                lang_ind += 1
        for en_char in en:
            if en_char not in en_voc2ind:
                en_voc2ind[en_char] = en_ind
                en_ind2voc[en_ind] = en_char
                en_ind += 1
                
    en_voc2ind[PAD] = en_ind
    lang_voc2ind[PAD] = lang_ind
    en_ind2voc[en_ind] = PAD
    lang_ind2voc[lang_ind] = PAD
    
    return lang_voc2ind, lang_ind2voc, en_voc2ind, en_ind2voc

In [296]:
# Preprocessing functions
def tokenize_data(data, voc2ind):
    return [voc2ind[char] for char in data]

def detokenize_and_depad_data(data, ind2voc, pad_val=0):
    res = []
    for ind in data:
        if ind.item() == pad_val:
            break
        res.append(ind2voc[ind.item()])
    return res

def pad_data(data, seq_len, pad_val=0):
    for _ in range(len(data), seq_len):
        data.append(pad_val)
    return data

In [297]:
class TransliterateDataset(torch.utils.data.Dataset):
    
    def __init__(self, data, lang_voc2ind, en_voc2ind, seq_len):
        super(TransliterateDataset, self).__init__()
        
        # Tokenize and pad data
        lang_data = []
        en_data = []
        for lang, en in data:
            lang_tokens = tokenize_data(lang, lang_voc2ind)
            lang_data.append(pad_data(lang_tokens, seq_len, len(lang_voc2ind) - 1))
            en_tokens = tokenize_data(en, en_voc2ind)
            en_data.append(pad_data(en_tokens, seq_len, len(en_voc2ind) - 1))
            
        self.language_data = lang_data
        self.english_data = en_data
        self.sequence_length = seq_len

    def __len__(self):
        return len(self.english_data)
        
    def __getitem__(self, idx):
        # Return the data and label at idx
        return torch.LongTensor(self.english_data[idx]), torch.LongTensor(self.language_data[idx])
    
    def get_random_sample(self):
        return self.__getitem__(np.random.randint(self.__len__()))

In [298]:
class TransliterateNet(nn.Module):
    
    def __init__(self, en_vocab_size, lang_vocab_size, feature_size):
        super(TransliterateNet, self).__init__()
        self.en_vocab_size = en_vocab_size
        self.lang_vocab_size = lang_vocab_size
        self.feature_size = feature_size

        self.encoder = nn.Embedding(self.en_vocab_size, self.feature_size)
        self.rnn = nn.GRU(self.feature_size, self.feature_size, num_layers = 2, batch_first=True)
        self.decoder = nn.Linear(self.feature_size, self.lang_vocab_size)
        
        #self.decoder.weight = self.encoder.weight
        #self.decoder.bias.data.zero_()
        
        self.best_accuracy = -1
    
    def forward(self, x, hidden_state=None):
        x = self.encoder(x)
        x, hidden_state = self.rnn(x, hidden_state)
        x = self.decoder(x)
        return x, hidden_state

    # This defines the function that gives a probability distribution and implements the temperature computation.
    def inference(self, x, hidden_state=None, temperature=1):
        x = x.view(-1, 1)
        x, hidden_state = self.forward(x, hidden_state)
        x = x.view(1, -1)
        x = x / max(temperature, 1e-20)
        x = F.softmax(x, dim=1)
        return x, hidden_state

    # Predefined loss function
    def loss(self, prediction, label, reduction='mean'):
        loss_val = F.cross_entropy(prediction.view(-1, self.lang_vocab_size), label.view(-1), reduction=reduction)
        return loss_val

    # Saves the current model
    def save_model(self, file_path, num_to_keep=1):
        pt_util.save(self, file_path, num_to_keep)

    # Saves the best model so far
    def save_best_model(self, accuracy, file_path, num_to_keep=1):
        if accuracy > self.best_accuracy:
            self.save_model(file_path, num_to_keep)
            self.best_accuracy = accuracy

    def load_model(self, file_path):
        pt_util.restore(self, file_path)

    def load_last_model(self, dir_path):
        return pt_util.restore_latest(self, dir_path)

In [1]:
def repackage_hidden(h):
    # Wraps hidden states in new Tensors, to detach them from their history
    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

def train(model, device, optimizer, train_loader, lr, epoch, log_interval):
    model.train()
    losses = []
    hidden = None
    for batch_idx, (data, label) in enumerate(tqdm.tqdm(train_loader)):
        data, label = data.to(device), label.to(device)
        # Separates the hidden state across batches.
        # Otherwise the backward would try to go all the way to the beginning every time
        if hidden is not None:
            hidden = repackage_hidden(hidden)
        optimizer.zero_grad()
        output, hidden = model(data)
        pred = output.max(-1)[1]
        loss = model.loss(output, label)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    return np.mean(losses)


def test(model, device, test_loader, en_ind2voc, lang_ind2voc):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        hidden = None
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.to(device), label.to(device)
            output, hidden = model(data, hidden)
            test_loss += model.loss(output, label, reduction='mean').item()
            pred = output.max(-1)[1]
            correct_mask = pred.eq(label.view_as(pred))
            num_correct = correct_mask.sum().item()
            correct += num_correct
            # Comment this out to avoid printing test results
            if batch_idx % 10 == 0:
                print('Input\t%s\nGT\t%s\npred\t%s\n\n' % (
                    detokenize_and_depad_data(data[0], en_ind2voc, len(en_ind2voc) - 1),
                    detokenize_and_depad_data(label[0], lang_ind2voc, len(lang_ind2voc) - 1),
                    detokenize_and_depad_data(pred[0], lang_ind2voc, len(lang_ind2voc) - 1)))

    test_loss /= len(test_loader)
    test_accuracy = 100. * correct / (len(test_loader.dataset) * test_loader.dataset.sequence_length)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset) * test_loader.dataset.sequence_length,
        100. * correct / (len(test_loader.dataset) * test_loader.dataset.sequence_length)))
    return test_loss, test_accuracy

In [ ]:
def main():
    BATCH_SIZE = 512
    FEATURE_SIZE = 128
    TEST_BATCH_SIZE = 256
    EPOCHS = 20
    LEARNING_RATE = 0.005
    MOMENTUM = 0.9
    WEIGHT_DECAY = 0.000005
    USE_CUDA = True
    PRINT_INTERVAL = 10
    OUT_PATH = 'out/'
    LOG_PATH = OUT_PATH + 'logs/log.pkl'
    
    train_data, test_data, seq_len = prepare_data(DATA_PATH)
    lang_voc2ind, lang_ind2voc, en_voc2ind, en_ind2voc = create_dictionaries(train_data + test_data)
    train_dataset = TransliterateDataset(train_data, lang_voc2ind, en_voc2ind, seq_len)
    test_dataset = TransliterateDataset(test_data, lang_voc2ind, en_voc2ind, seq_len)
    
    use_cuda = USE_CUDA and torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    print('Using device', device)
    
    num_workers = multiprocessing.cpu_count()
    print('num workers:', num_workers)
    kwargs = {'num_workers': num_workers,
              'pin_memory': True} if use_cuda else {}
    
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, drop_last=True,
                                               shuffle=False, **kwargs)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TEST_BATCH_SIZE, drop_last=True,
                                              shuffle=False, **kwargs)
    
    model = TransliterateNet(len(en_voc2ind), len(lang_voc2ind), FEATURE_SIZE).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    #optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    start_epoch = model.load_last_model(OUT_PATH + 'checkpoints')

    train_losses, test_losses, test_accuracies = pt_util.read_log(LOG_PATH, ([], [], []))
    test_loss, test_accuracy = test(model, device, test_loader, en_ind2voc, lang_ind2voc)

    test_losses.append((start_epoch, test_loss))
    test_accuracies.append((start_epoch, test_accuracy))
    
    try:
        for epoch in range(start_epoch, EPOCHS + 1):
            lr = LEARNING_RATE * np.power(0.25, (int(epoch / 6)))
            train_loss = train(model, device, optimizer, train_loader, lr, epoch, PRINT_INTERVAL)
            test_loss, test_accuracy = test(model, device, test_loader, en_ind2voc, lang_ind2voc)
            train_losses.append((epoch, train_loss))
            test_losses.append((epoch, test_loss))
            test_accuracies.append((epoch, test_accuracy))
            #pt_util.write_log(LOG_PATH, (train_losses, test_losses, test_accuracies))
            #model.save_best_model(test_accuracy, DATA_PATH + 'checkpoints/%03d.pt' % epoch)

    except KeyboardInterrupt as ke:
        print('Interrupted')
    except:
        import traceback
        traceback.print_exc()
    finally:
        #print('Saving final model')
        #model.save_model(DATA_PATH + 'checkpoints/%03d.pt' % epoch, 0)
        ep, val = zip(*train_losses)
        pt_util.plot(ep, val, 'Train loss', 'Epoch', 'Error')
        ep, val = zip(*test_losses)
        pt_util.plot(ep, val, 'Test loss', 'Epoch', 'Error')
        ep, val = zip(*test_accuracies)
        pt_util.plot(ep, val, 'Test accuracy', 'Epoch', 'Error')
        return model, en_voc2ind, lang_ind2voc, device
    
final_model, en_voc2ind, lang_ind2voc, device = main()

Using device cpu
num workers: 48
Input	[]
GT	[]
pred	['ஜ', 'ஜ', 'ஜ', 'ங', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம']


Input	['n', 'e', 't', 'r', 'i']
GT	['ந', 'ெ', 'ற', '்', 'ற', 'ி']
pred	['ம', 'ண', 'ண', 'ங', 'ங', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம']


Input	['k', 'a', 'a', 'r', 'a', 'n', 'a', 't', 't', 'a', 'i', 'p']
GT	['க', 'ா', 'ர', 'ண', 'த', '்', 'த', 'ை', 'ப', '்']
pred	['ம', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ங', 'ஹ', 'ஹ', 'ஹ', 'ஹ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம']


Input	['v', 'a', 'a', 'g', 'a', 'n', 'a']
GT	['வ', 'ா', 'க', 'ன']
pred	['ம', 'ம', 'ஜ', 'ஹ', 'ஹ', 'ஹ', 'ஹ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ஜ', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம', 'ம']



Test set: Average loss: 3.8472, Accuracy: 674/239760 (0%)



  1%|▏         | 1/75 [00:00<00:13,  5.50it/s]

Train Epoch: 0 [0/38723 (0%)]	Loss: 3.858480


 16%|█▌        | 12/75 [00:02<00:11,  5.43it/s]

Train Epoch: 0 [5120/38723 (13%)]	Loss: 2.820517


...

 16%|█▌        | 12/75 [00:02<00:10,  5.73it/s]

Train Epoch: 14 [5120/38723 (13%)]	Loss: 1.619100


 29%|██▉       | 22/75 [00:03<00:09,  5.43it/s]

Train Epoch: 14 [10240/38723 (27%)]	Loss: 2.334424


 32%|███▏      | 24/75 [00:04<00:09,  5.51it/s]

In [281]:
def generate_transliteration(model, word, en_voc2ind, lang_ind2voc, seq_len=30):
    transliteration = []
    hidden = None
    
    en_tokens = tokenize_data(word, en_voc2ind)
    en_data = torch.LongTensor(pad_data(en_tokens, seq_len))
    
    for c in torch.LongTensor(en_data):
        x, hidden = model.inference(c, hidden)
        transliteration.append(torch.argmax(x))
    
    return detokenize_and_depad_data(transliteration, lang_ind2voc, len(lang_ind2voc) - 1)

In [282]:
print(generate_transliteration(final_model, 'farm', en_voc2ind, lang_ind2voc)) # should ideally say 'ஃபார்ம்'

['ப', 'ா', 'ர', '்', 'க', '்', 'ப', '்', 'ப', '்', 'ப', 'ெ', 'ப', 'ெ', 'ப', 'ெ', 'ஃ', 'ெ', 'ஃ', 'ெ', 'ஃ', 'ெ', 'ஃ', 'ெ', 'ஃ', 'ெ', 'ஃ', 'ெ', 'ஃ', 'ெ']
